In [ ]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

# Import API key
weather_api_key = "d0613004fa405a4f72a41754bc37747c"

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [ ]:

# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
url = "http://api.openweathermap.org/data/2.5/forecast?id=524901&APPID="
target_url = url + weather_api_key
target_url

In [ ]:
weather_response = requests.get(target_url)
weather_json = weather_response.json()
#print(json.dumps(weather_json, indent=4, sort_keys=True))

In [ ]:
#print(weather_json['city']['name'])
print(weather_json['city']['name'])

In [ ]:

# set up lists to hold reponse info
cityname = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []
count = 0

#  response = requests.get(query_url + city).json()
    
# Loop through the list of cities and perform a request for data on each

for city in cities:
    weather_response = requests.get(target_url + "&q=" + city + "&units=Imperial")
    weather_json = weather_response.json()
    try:
        cityname.append(weather_json['city']['name'])
        cloudiness.append(weather_json['list'][0]['clouds']['all'])
        country.append(weather_json['city']['country'])
        date.append(weather_json['list'][0]['dt'])
        humidity.append(weather_json['list'][0]['main']['humidity'])
        lat.append(weather_json['city']['coord']['lat'])
        lng.append(weather_json['city']['coord']['lon'])
        max_temp.append(weather_json['list'][0]['main']['temp_max'])
        wind_speed.append(weather_json['list'][0]['wind']['speed'])
        count = count + 1
        print(f"Processing record {count} of Set 1 | {city}")
    except:
        print("Record not found") 
        pass


In [ ]:
# Convert array of JSONs into Pandas DataFrame

weather_dict = {"City": cityname,"Latitude": lat, "Longitude":lng,
                "Max Temp":max_temp,"Humidity":humidity,"Cloudiness":cloudiness,
                "Wind Speed":wind_speed,"Country":country, "Date":date}
weather_data = pd.DataFrame(weather_dict)
weather_data.head()

In [ ]:
# Export the City_Data into a csv
weather_data.to_csv(r'output_data\my_weather_data.csv')

In [ ]:
# Scatter plot for latitude vs. temperature
scatlat = weather_data["Latitude"]
scattemp = weather_data["Max Temp"]

plt.scatter(scatlat,scattemp)
plt.title('City Latitude vs. Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.savefig('output_data/scatplotlatitudemaxtemp')
plt.show()
print("This scatter plot shows how temperature varies based on Latitude.")

# Scatter plot for latitude vs. humidity
scathumid = weather_data["Humidity"]

plt.scatter(scatlat,scathumid)
plt.title('City Latitude vs. Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.savefig('output_data/scatplotlatitudehumidity')
plt.show()
print("This scatter plot shows how temperature correlates with humidity.")

# Scatter plot for latitude vs. cloudiness
scatcloud = weather_data["Cloudiness"]

plt.scatter(scatlat,scatcloud)
plt.title('City Latitude vs. Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.savefig('output_data/scatplotlatitudecloudiness')
plt.show()
print("This scatter plot analyzes if any correlation exists between cloudiness and    city latitude.")

# Scatter plot for latitude vs. wind speed
scatwind = weather_data["Wind Speed"]

plt.scatter(scatlat,scatwind)
plt.title('City Latitude vs. Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.savefig('output_data/scatplotlatitudewindspeed')
plt.show()

In [ ]:
def plot_linear_regression(x_values, y_values, title, text_coordinates):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,text_coordinates,fontsize=14,color="red")
    plt.xlabel("Latitude")
    plt.ylabel(title)
    print(f"The r-squared is: {rvalue}")
    plt.show()

In [ ]:
northlat_df = weather_data.loc[weather_data["Latitude"] >= 0,:]
southlat_df = weather_data.loc[weather_data["Latitude"] < 0,:]


# Temperature (F) vs. Latitude

In [ ]:
x_values = northlat_df["Latitude"]
y_values = northlat_df["Max Temp"]
plt.title("Northern Hemisphere - Temperature (F) vs Latitude")
plot_linear_regression(x_values,y_values,'Max Temp',(6,30))

plt.savefig('output_data/Northern Hemisphere - Temperature (F) vs Latitude')

x_values = southlat_df["Latitude"]
y_values = southlat_df["Max Temp"]
plt.title("Southern Hemisphere - Temperature (F) vs Latitude")
plot_linear_regression(x_values,y_values,'Max Temp',(6,30))

plt.savefig('output_data/Southern Hemisphere - Temperature (F) vs Latitude')

# Humidity (%) vs. Latitude

In [ ]:
x_values = northlat_df["Latitude"]
y_values = northlat_df["Humidity"]
plt.title("Northern Hemisphere - Humidity (%) vs Latitude")
plot_linear_regression(x_values,y_values,'Humidity',(6,30))

plt.savefig("output_data/Northern Hemisphere - Humidity (%) vs Latitude")

x_values = southlat_df["Latitude"]
y_values = southlat_df["Humidity"]
plt.title("Southern Hemisphere - Humidity (%) vs Latitude")
plot_linear_regression(x_values,y_values,'Humidity',(6,30))

plt.savefig('output_data/Southern Hemisphere - Humidity (%) vs Latitude')

# Cloudiness (%) vs. Latitude

In [ ]:
x_values = northlat_df["Latitude"]
y_values = northlat_df["Cloudiness"]
plt.title("Northern Hemisphere - Cloudiness (%) vs Latitude")
plot_linear_regression(x_values,y_values,'Cloudiness',(6,30))

plt.savefig("output_data/Northern Hemisphere - Cloudiness (%) vs Latitude")

x_values = southlat_df["Latitude"]
y_values = southlat_df["Cloudiness"]
plt.title("Southern Hemisphere - Cloudiness (%) vs Latitude")
plot_linear_regression(x_values,y_values,'Cloudiness',(6,30))

plt.savefig("output_data/Southern Hemisphere - Cloudiness (%) vs Latitude")

# Wind Speed (mph) vs. Latitude

In [ ]:
x_values = northlat_df["Latitude"]
y_values = northlat_df["Wind Speed"]
plt.title("Northern Hemisphere - Wind Speed (mph) vs Latitude")
plot_linear_regression(x_values,y_values,'Wind Speed',(6,30))

plt.savefig("output_data/Northern Hemisphere - Wind Speed (mph) vs Latitude")

x_values = southlat_df["Latitude"]
y_values = southlat_df["Wind Speed"]
plt.title("Southern Hemisphere - Wind Speed (mph) vs Latitude")
plot_linear_regression(x_values,y_values,'Wind Speed',(6,30))

plt.savefig("output_data/Southern Hemisphere - Wind Speed (mph) vs Latitude")
